In [2]:
# Load Data into a SQL Database

In [2]:
# Before writing SQL queries, we need to load the cleaned CSV files into a SQLite database. 
# You can do this using SQLite (a lightweight database).

# Here’s how to load data into SQLite using Python:
import sqlite3
import pandas as pd
import os

# Connect to SQLite database (or create one if it doesn't exist)
try:
    conn = sqlite3.connect("fetch_data.db")
    cursor = conn.cursor()
except sqlite3.Error as e:
    print(f"Error connecting to SQLite: {e}")
    exit(1)

# File paths
products_file = r"c:\Users\al_mu\GitHub\Fetch\Fetch---Data-Analyst\data\cleansed\products_cleaned.csv"
transactions_file = r"c:\Users\al_mu\GitHub\Fetch\Fetch---Data-Analyst\data\cleansed\transactions_cleaned.csv"
users_file = r"c:\Users\al_mu\GitHub\Fetch\Fetch---Data-Analyst\data\cleansed\users_cleaned.csv"

# Check if files exist
if not os.path.exists(products_file):
    print(f"Error: {products_file} does not exist.")
elif not os.path.exists(transactions_file):
    print(f"Error: {transactions_file} does not exist.")
elif not os.path.exists(users_file):
    print(f"Error: {users_file} does not exist.")
else:
    try:
        # Load cleaned CSVs into DataFrames
        products_df = pd.read_csv(products_file)
        transactions_df = pd.read_csv(transactions_file)
        users_df = pd.read_csv(users_file)

        # Check if any DataFrame is empty
        if products_df.empty or transactions_df.empty or users_df.empty:
            print("Warning: One or more of the CSV files are empty.")
        else:
            # Save DataFrames to SQLite
            products_df.to_sql("products", conn, if_exists="replace", index=False)
            transactions_df.to_sql("transactions", conn, if_exists="replace", index=False)
            users_df.to_sql("users", conn, if_exists="replace", index=False)

            print("Data successfully loaded into SQLite database.")
            
            # Verify that data has been loaded correctly by querying the tables
            products_count = cursor.execute("SELECT COUNT(*) FROM products").fetchone()[0]
            transactions_count = cursor.execute("SELECT COUNT(*) FROM transactions").fetchone()[0]
            users_count = cursor.execute("SELECT COUNT(*) FROM users").fetchone()[0]

            print(f"Products table contains {products_count} records.")
            print(f"Transactions table contains {transactions_count} records.")
            print(f"Users table contains {users_count} records.")

    except Exception as e:
        print(f"Error while loading data into SQLite: {e}")
    finally:
        # Close the connection
        conn.close()


Data successfully loaded into SQLite database.
Products table contains 845337 records.
Transactions table contains 49829 records.
Users table contains 100000 records.


In [12]:
# connect to db
conn = sqlite3.connect("fetch_data.db")
cursor = conn.cursor()

In [13]:

# 1.a) What are the top 5 brands by receipts scanned among users 21 and over?
query = """ 
SELECT p.BRAND, COUNT(t.RECEIPT_ID) AS receipts_scanned
FROM transactions t
JOIN users u ON t.USER_ID = u.ID
JOIN products p ON t.BARCODE = p.BARCODE
WHERE (strftime('%Y', 'now') - strftime('%Y', u.BIRTH_DATE)) >= 21
GROUP BY p.BRAND
ORDER BY receipts_scanned DESC
LIMIT 5
"""  

result = pd.read_sql_query(query, conn)
print(result)



             BRAND  receipts_scanned
0          Unknown                 6
1      NERDS CANDY                 6
2             DOVE                 6
3          TRIDENT                 4
4  SOUR PATCH KIDS                 4


In [14]:
# 1.b) What are the top 5 brands by sales among users that have had their account for at least six months?

query = """ 
SELECT p.BRAND, SUM(t.FINAL_SALE) AS total_sales
FROM transactions t
JOIN users u ON t.USER_ID = u.ID
JOIN products p ON t.BARCODE = p.BARCODE
WHERE (strftime('%Y', 'now') - strftime('%Y', u.CREATED_DATE)) >= 0.5
GROUP BY p.BRAND
ORDER BY total_sales DESC
LIMIT 5;
"""  
result = pd.read_sql_query(query, conn)
print(result)


         BRAND  total_sales
0          CVS        72.00
1      TRIDENT        46.72
2         DOVE        42.88
3  COORS LIGHT        34.96
4      Unknown        16.65


In [15]:
# 1.c) What is the percentage of sales in the Health & Wellness category by generation?

query = """ 
SELECT
    CASE
        WHEN (strftime('%Y', 'now') - strftime('%Y', u.BIRTH_DATE)) BETWEEN 18 AND 25 THEN 'Gen Z'
        WHEN (strftime('%Y', 'now') - strftime('%Y', u.BIRTH_DATE)) BETWEEN 26 AND 40 THEN 'Millennials'
        WHEN (strftime('%Y', 'now') - strftime('%Y', u.BIRTH_DATE)) BETWEEN 41 AND 56 THEN 'Gen X'
        ELSE 'Boomers'
    END AS generation,
    SUM(CASE WHEN p.CATEGORY_1 = 'Health & Wellness' THEN t.FINAL_SALE ELSE 0 END) / SUM(t.FINAL_SALE) * 100 AS percentage_sales
FROM transactions t
JOIN users u ON t.USER_ID = u.ID
JOIN products p ON t.BARCODE = p.BARCODE
GROUP BY generation;
"""  

result = pd.read_sql_query(query, conn)
print(result)

    generation  percentage_sales
0      Boomers         35.213098
1        Gen X         36.860304
2  Millennials         36.376433


In [16]:
# 2.a) Who are Fetch’s power users?
query = """ 
SELECT u.ID, SUM(t.FINAL_SALE) AS total_sales
FROM transactions t
JOIN users u ON t.USER_ID = u.ID
GROUP BY u.ID
ORDER BY total_sales DESC
LIMIT 10;  -- Top 10 power users
"""  

result = pd.read_sql_query(query, conn)
print(result)



                         ID  total_sales
0  643059f0838dd2651fb27f50        75.99
1  62ffec490d9dbaff18c0a999        52.28
2  5f4c9055e81e6f162e3f6fa8        37.96
3  5d191765c8b1ba28e74e8463        34.96
4  6351760a3a4a3534d9393ecd        27.74
5  64dd9170516348066e7c4006        26.52
6  62c09104baa38d1a1f6c260e        20.28
7  61a58ac49c135b462ccddd1c        19.92
8  6661ed1e7c0469953bfc76c4        18.60
9  5b441360be53340f289b0795        18.32


In [17]:
#  2.b) Which is the leading brand in the Dips & Salsa category?
query = """ 
SELECT p.BRAND, SUM(t.FINAL_SALE) AS total_sales
FROM transactions t
JOIN products p ON t.BARCODE = p.BARCODE
WHERE p.CATEGORY_2 = 'Dips & Salsa'
GROUP BY p.BRAND
ORDER BY total_sales DESC
LIMIT 1;
"""  

result = pd.read_sql_query(query, conn)
print(result)



      BRAND  total_sales
0  TOSTITOS       260.99


In [9]:
#  1.c) At what percent has Fetch grown year over year?
query = """ 
WITH current_year_sales AS (
    SELECT COALESCE(SUM(t.FINAL_SALE), 0) AS total_sales
    FROM transactions t
    WHERE strftime('%Y', t.PURCHASE_DATE) = strftime('%Y', 'now')  -- Current year
),
previous_year_sales AS (
    SELECT COALESCE(SUM(t.FINAL_SALE), 0) AS total_sales
    FROM transactions t
    WHERE strftime('%Y', t.PURCHASE_DATE) = strftime('%Y', 'now', '-1 year')  -- Previous year
)
SELECT 
    current_year_sales.total_sales AS current_year_sales,
    previous_year_sales.total_sales AS previous_year_sales,
    CASE 
        WHEN previous_year_sales.total_sales > 0 THEN 
            ((current_year_sales.total_sales - previous_year_sales.total_sales) / previous_year_sales.total_sales) * 100
        ELSE
            NULL  -- If no sales in previous year, return NULL for growth percentage
    END AS growth_percentage
FROM current_year_sales, previous_year_sales;

"""  

result = pd.read_sql_query(query, conn)
print(result)



   current_year_sales  previous_year_sales  growth_percentage
0                   0            171182.79             -100.0


In [10]:
# -- becuse we don't have data for this year so we will compare the last year with the year before it.

query = """
WITH previous_year_sales AS (
    SELECT COALESCE(SUM(t.FINAL_SALE), 0) AS total_sales
    FROM transactions t
    WHERE strftime('%Y', t.PURCHASE_DATE) = strftime('%Y', 'now', '-1 year')  -- Previous year
),
two_years_ago_sales AS (
    SELECT COALESCE(SUM(t.FINAL_SALE), 0) AS total_sales
    FROM transactions t
    WHERE strftime('%Y', t.PURCHASE_DATE) = strftime('%Y', 'now', '-2 years')  -- Two years ago
)
SELECT 
    previous_year_sales.total_sales AS previous_year_sales,
    two_years_ago_sales.total_sales AS two_years_ago_sales,
    CASE 
        WHEN two_years_ago_sales.total_sales > 0 THEN 
            ((previous_year_sales.total_sales - two_years_ago_sales.total_sales) / two_years_ago_sales.total_sales) * 100
        ELSE
            NULL  -- If no sales two years ago, return NULL for growth percentage
    END AS growth_percentage
FROM previous_year_sales, two_years_ago_sales;

"""

result = pd.read_sql_query(query, conn)
print(result)


   previous_year_sales  two_years_ago_sales growth_percentage
0            171182.79                    0              None


In [11]:
# Close the connection
conn.close()